## Letterboxd Data exploration

The Data used in this project can be found here: https://www.kaggle.com/datasets/gsimonx37/letterboxd
It will be placed in the directory "letterboxd", that is added to the .gitignore because of file size limitations in GitHub.

In [1]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
#load data
actors = pd.read_csv("letterboxd/actors.csv")
countries = pd.read_csv("letterboxd/countries.csv")
crew = pd.read_csv("letterboxd/crew.csv")
genres = pd.read_csv("letterboxd/genres.csv")
movies = pd.read_csv("letterboxd/movies.csv")
languages = pd.read_csv("letterboxd/languages.csv")
releases = pd.read_csv("letterboxd/releases.csv")
studios = pd.read_csv("letterboxd/studios.csv")
themes = pd.read_csv("letterboxd/themes.csv")


# Actors 
Contains name of actor/actress and the corresponsing movie in the form of "id"

In [11]:
#check data
print(actors.head())

#check datatypes
print(actors.dtypes)

        id                name
0  1000001       Margot Robbie
1  1000001        Lewis Easter
2  1000001  Onyemachi Ejimofor
3  1000001     Cameron Everitt
4  1000001   Luke Field-Wright
id       int64
name    object
dtype: object


# Countries
Contains country and the corresponsing movie in the form of "id"

In [12]:
#check data
print(countries.head())

#check datatypes
print(countries.dtypes)

        id      country
0  1000001           UK
1  1000001          USA
2  1000002  South Korea
3  1000003          USA
4  1000004      Germany
id          int64
country    object
dtype: object
